In [77]:
import pandas as pd
import yfinance as yf

from datetime import date
from dateutil.relativedelta import relativedelta
import json
import time

In [78]:
with open('../data/tickers.json') as f:
    tickers = json.load(f)

Get ticker symbols to the list

In [79]:
tickers = [ticker for ticker in tickers.keys()]

Set date range

In [80]:
end_date = date.today()
start_date = end_date - relativedelta(months=6)

In [ ]:
# tickers = ['AAPL', 'MSFT']

# df_list_close = []
# df_list_volume = []

# for ticker in tickers:

#     data = yf.download(ticker, start='2024-01-01', end='2025-01-01', progress=False)
#     # Dodavanje kolone za ticker
#     data['ticker'] = ticker
    
#     # Izračunavanje postotne promene za close_price i volume
#     data['close_price_change'] = data['Close'].pct_change()   # Procenat promene cene
#     data['volume_change'] = data['Volume'].pct_change()   # Procenat promene volumena

#     # Dodavanje samo potrebnih kolona u listu, koristeći indeks umesto 'date' kolone
#     df_list_close.append(data[['close_price_change']].rename(columns={'close_price_change': f'{ticker}_close_change'}))
#     df_list_volume.append(data[['volume_change']].rename(columns={'volume_change': f'{ticker}_volume_change'}))

# # Kombinovanje svih DataFrame-ova za close_price_change i volume_change
# final_close = pd.concat(df_list_close, axis=1)
# final_volume = pd.concat(df_list_volume, axis=1)

# # Spajanje close_price_change i volume_change DataFrame-a
# final_df = pd.concat([final_close, final_volume], axis=1)

# # Resetovanje indeksa da bi 'date' postao obična kolona
# final_df.reset_index(inplace=True)

# # Postavljanje tačnog redosleda kolona
# final_df = final_df[['Date', 'AAPL_close_change', 'AAPL_volume_change', 'MSFT_close_change', 'MSFT_volume_change']]

# # Prikazivanje rezultata
# final_df


Price,Date,AAPL_close_change,AAPL_volume_change,MSFT_close_change,MSFT_volume_change
Ticker,,,,,
0,2024-01-02,NaN,NaN,NaN,NaN
1,2024-01-03,-0.007487,-0.291848,-0.000728,-0.086113
2,2024-01-04,-0.012700,0.232290,-0.007178,-0.094526
3,2024-01-05,-0.004013,-0.134479,-0.000516,0.004091
4,2024-01-08,0.024175,-0.050700,0.018872,0.102301
...,...,...,...,...,...
247,2024-12-24,0.011478,-0.431342,0.009374,-0.625924
248,2024-12-26,0.003176,0.172260,-0.002777,0.143723
249,2024-12-27,-0.013242,0.555059,-0.017302,1.211040


In [ ]:
# tickers = ['AAPL', 'MSFT']

data = yf.download(tickers, start=start_date, end=end_date, progress=False)

if isinstance(data.columns, pd.MultiIndex):
    data.columns.names = [None, None]  # uklanja imena nivoa
    data.columns = ['_'.join(col).strip() for col in data.columns.values]

all_data = []

for ticker in tickers:

    print(f'Downloading {ticker}... waiting 10 seconds.')
    time.sleep(10)

    subset = data[[f'{col}_{ticker}' for col in ['Open', 'Close', 'High', 'Low', 'Volume']]].copy()
    subset.columns = ['Open', 'Close', 'High', 'Low', 'Volume']
    subset['Date'] = data.index
    subset['ticker'] = ticker
    all_data.append(subset[['Date', 'ticker', 'Open', 'Close', 'High', 'Low', 'Volume']])

final_df = pd.concat(all_data, ignore_index=True)
final_df = final_df.dropna()




1 Failed download:
['ALVOW']: YFInvalidPeriodError("ALVOW: Period 'max' is invalid, must be of the format 1d, 5d, etc.")


In [ ]:
final_df.to_csv('data/stock_data.csv', index=False)